In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

In [3]:
train_dataset = MNIST(True)
test_dataset = MNIST(False)

In [4]:
train_x, train_y = train_dataset.train_data.numpy(), train_dataset.train_labels
test_x, test_y = test_dataset.test_data.numpy(), test_dataset.test_labels
train_y2 = to_one_hot(train_y, 10).numpy()
test_y2 = to_one_hot(test_y, 10).numpy()
train_y = train_y.numpy()
test_y = test_y.numpy()

In [5]:
train_data = FromNumpy(train_x[:, None, :, :], [train_y2, train_y], batch_size=256)
test_data = FromNumpy(test_x[:, None, :, :], [test_y2, test_y], batch_size=256)

In [6]:
class Auxiliary(nn.Module):
    def __init__(self, f):
        super().__init__()
        self.f = f
        self.store = None
    
    def forward(self, x):
        # Store the result
        # Return the origin value
        self.store = self.f(x)
        return x 

In [29]:
model = builder([
    nn.Conv2d(1, 20, 5, 1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(20, 50, 5, 1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    Flatten(),
    nn.Linear(4*4*50, 500),
    nn.ReLU(),
    nn.Linear(500, 10),
    Auxiliary(nn.Sigmoid()),
    nn.Softmax(1)
])

In [30]:
runner = Runner(model, train_data, test_data, 'adam', ['bce', 'nll'])

In [31]:
runner.fit(4)

  Epochs    train_loss    train_acc      val_loss      val_acc    
    1*       -0.7955        0.9094       -0.9568        0.9788    


    2        -0.9716        0.9856       -0.9804        0.9896    


    3        -0.9831        0.9919       -0.9707        0.9872    


    4        -0.9848        0.9925       -0.9856        0.9930    



In [32]:
model = runner.model

In [45]:
data = next(iter(train_data))
x = data[0][0]
x = x[None, :, :, :]
out1 = model(x.cuda())
out2 = get_aux_out(model)[0]
print(out1)
print(out2)
print(out1*out2)


tensor([[6.6702e-14, 6.2851e-13, 6.6820e-15, 1.0777e-17, 5.3743e-13, 2.1907e-12,
         1.0000e+00, 1.9527e-13, 5.4591e-16, 7.4643e-18]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
tensor([[4.4454e-07, 4.1887e-06, 4.4532e-08, 7.1821e-11, 3.5817e-06, 1.4600e-05,
         1.0000e+00, 1.3014e-06, 3.6382e-09, 4.9746e-11]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor([[2.9651e-20, 2.6327e-18, 2.9757e-22, 7.7399e-28, 1.9249e-18, 3.1985e-17,
         1.0000e+00, 2.5413e-19, 1.9861e-24, 3.7132e-28]], device='cuda:0',
       grad_fn=<MulBackward0>)
